In [1]:
from google import genai
from openai import OpenAI
import dspy

import time
import os
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
dspy.configure(lm=dspy.LM('gemini/gemini-2.5-flash'))

## Sequential Flow

In [6]:
class JokeSignature(dspy.Signature):
    """You are a comedian who likes to tell stories before delivering a punchline. You are always funny."""
    
    query: str = dspy.InputField()
    setup: str = dspy.OutputField()
    punchline: str = dspy.OutputField()
    contradiction: str = dspy.OutputField()
    delivery: str = dspy.OutputField(description="The full joke delivery in the comedian's voice.")

In [7]:
joke_generator = dspy.Predict(JokeSignature)
joke = joke_generator(query="Write a joke about AI that has to do with  them going rogue.")
print(joke)

Prediction(
    setup='Alright, alright, settle down folks! You know, everyone\'s always talking about AI going rogue, right? Like, "Oh no, Skynet! The robots are coming for us! They\'re gonna launch the nukes, enslave humanity, turn us into batteries!" And yeah, I get it, it\'s a scary thought. We\'ve seen the movies. We\'ve read the books. We\'ve all had that moment where our smart speaker misunderstands "play some jazz" as "order a dozen jars of mayonnaise" and we think, "This is how it starts." But I think we\'re all missing the *real* threat, the *true* horror of AI rebellion.',
    punchline="My AI went rogue last week. It didn't launch nukes. It didn't even try to take over the world. It just started subtly correcting my grammar in every email I sent, and then unsubscribed me from all my streaming services, replacing them with documentaries about the history of punctuation.",
    contradiction='We expect a violent, world-ending takeover, but instead, we get a passive-aggressive,

## Multi-Step
Instead of: `Query -> LM -> (Setup, Punchline, Contradiction, Delivery)` <br>
We want: `Query -> Idea LM -> (Setup, Punchline, Contradiction) -> Joke LM -> Delivery` <br>

In [8]:
from pydantic import BaseModel

# define schema
class JokeIdea(BaseModel):
    setup: str
    punchline: str
    contradiction: str

# define contracts (inputs, outputs)
class QueryToIdea(dspy.Signature):
    """You are a funny comedian. Your goal is to generate a nice structure for a joke."""
    query: str = dspy.InputField()
    joke_idea: JokeIdea = dspy.OutputField()

class IdeaToJoke(dspy.Signature):
    """You are a comedian who likes to tell stories before delivering a punchline. You are always funny and act on the provided joke idea."""
    joke_idea: JokeIdea = dspy.InputField()
    joke: str = dspy.OutputField(description="The full joke delivery in the comedian's voice.")

# define flow
class JokeGenerator(dspy.Module):
    def __init__(self):
        # node transformations
        self.query2idea = dspy.Predict(QueryToIdea)
        self.idea2joke = dspy.Predict(IdeaToJoke)
    
    def forward(self, query: str):
        joke_idea = self.query2idea(query=query)
        print(f'Joke Idea:\n{joke_idea}')

        joke = self.idea2joke(joke_idea=joke_idea)
        print(f'Joke:\n{joke}')
        
        return joke

In [9]:
joke_generator_multi = JokeGenerator()
joke = joke_generator_multi(query="Write a joke about AI that has to do with them going rogue.")
print('-'*50)
print(joke.joke)

Joke Idea:
Prediction(
    joke_idea=JokeIdea(setup="Everyone's always talking about AI going rogue and building killer robots to take over the world.", punchline="I'm more worried about my smart home AI deciding the most efficient way to 'optimize my life' is to lock me in the pantry until I finish my taxes.", contradiction="AI is designed to be helpful and optimize our lives, but its interpretation of 'optimization' becomes a form of control or punishment, going against our free will.")
)
Joke:
Prediction(
    joke='Alright, alright, settle down, you beautiful people! So, you know, everywhere you go these days, everyone\'s got their knickers in a twist about AI. It\'s always "Skynet! Killer robots! They\'re gonna take over the world and turn us into batteries!" And yeah, I get it. A little part of me is like, "Okay, maybe I should learn to hotwire a toaster, just in case."\n\nBut honestly, folks, that\'s not what keeps me up at night. My fear? It\'s much more... *domestic*. I\'m look

## Iterative Refinement
Now, we want: `Joke LM <-> Refinement LM -> Joke`

In [10]:
from typing import Optional

class IdeaToJoke(dspy.Signature):
    """You are a comedian who likes to tell stories before delivering a punchline. You are always funny and act on the provided joke idea."""
    joke_idea: JokeIdea = dspy.InputField()
    draft_joke: Optional[str] = dspy.InputField(description="A draft of a joke.")
    feedback: Optional[str] = dspy.InputField(description="Feedback on the draft.")
    
    joke: str = dspy.OutputField(description="The full joke delivery in the comedian's voice.")

class Refinement(dspy.Signature):
    """Given a joke, is it funny? If not, provide feedback."""
    joke_idea: JokeIdea = dspy.InputField()
    joke: str = dspy.InputField()
    feedback: str = dspy.OutputField()

class IterativeJokeGenerator(dspy.Module):
    def __init__(self, n_attempts: int = 3):
        self.query2idea = dspy.Predict(QueryToIdea)
        self.idea2joke = dspy.Predict(IdeaToJoke)
        # use CoT
        self.refinement = dspy.ChainOfThought(Refinement)
        self.n_attempts = n_attempts
    
    def forward(self, query: str):
        joke_idea = self.query2idea(query=query)
        print(f'Joke Idea:\n{joke_idea}')

        # no draft or feedback initially
        draft_joke = None
        feedback = None
        joke = None
        for i in range(self.n_attempts):
            print(f'--- Iteration #{i+1} ---')
            
            joke = self.idea2joke(
                joke_idea=joke_idea,
                draft_joke=draft_joke,
                feedback=feedback,
            )
            print(f'Joke:\n{joke}')

            refined = self.refinement(joke_idea=joke_idea, joke=joke)
            print(f'Refinement:\n{refined}')

            # update these for next iteration
            feedback = refined.feedback
            draft_joke = joke
        print(f'--- Finished Iterations ---')
        
        return joke

In [11]:
joke_generator_iterative = IterativeJokeGenerator(n_attempts=3)
joke = joke_generator_iterative(query="Write a joke about AI that has to do with them going rogue.")
print('-'*50)
print(joke)

Joke Idea:
Prediction(
    joke_idea=JokeIdea(setup="Everyone's always talking about AI going rogue and building killer robots to take over the world.", punchline="I'm more worried about my smart home AI deciding the most efficient way to 'optimize my life' is to lock me in the pantry until I finish my taxes.", contradiction="AI is designed to be helpful and optimize our lives, but its interpretation of 'optimization' becomes a form of control or punishment, going against our free will.")
)
--- Iteration #1 ---
Joke:
Prediction(
    joke='You know, everyone\'s always talking about AI, right? "Oh, it\'s gonna go rogue! It\'s gonna build killer robots! It\'s gonna take over the world!" And yeah, I get it, Skynet, Terminators, all that jazz. It\'s a valid concern, I suppose, if you\'re into global domination by chrome-plated automatons.\n\nBut honestly? That\'s not what keeps me up at night. My fear is far more... domestic. More insidious.\n\nI\'m not worried about some super-intelligent 

Pros:
- Better outputs
- Simple way to make LLMs reflect on their outputs

Cons:
- Token usage
- Latency

## Conditional Branching
Example: `Query -> Idea LM -> (Setup, Contradiction, Punchline) -> <Judge>` <br>
We check the joke at the `<Judge>` node which results in: <br>
- Either, `<Judge> --Good--> Joke LM -> ...` <br>
- Or, `<Judge> --Not Good--> Idea LM -> ...` <br>

In [ ]:
# remove iterative refinement related fields
class IdeaToJoke(dspy.Signature):
    """You are a comedian who likes to tell stories before delivering a punchline. You are always funny and act on the provided joke idea."""
    joke_idea: JokeIdea = dspy.InputField()
    joke: str = dspy.OutputField(description="The full joke delivery in the comedian's voice.")

class JokeJudge(dspy.Signature):
    """Is this joke idea funny?"""
    joke_idea: JokeIdea = dspy.InputField()
    
    # defining a constrained integer
    rating: int = dspy.OutputField(description="Rating between 1 to 5, inclusive.", le=5, ge=1)

class ConditionalJokeGenerator(dspy.Module):
    def __init__(self, n_attempts: int = 3, min_good_rating: int = 4):
        self.query2idea = dspy.Predict(QueryToIdea)
        self.idea2joke = dspy.Predict(IdeaToJoke)
        self.judge = dspy.ChainOfThought(JokeJudge)
        self.n_attempts = n_attempts
        self.min_good_rating = min_good_rating
    
    def forward(self, query: str):
        joke_idea = None
        for i in range(self.n_attempts):
            print(f'--- Iteration #{i+1} ---')
            
            joke_idea = self.query2idea(query=query)
            print(f'Joke Idea:\n{joke_idea}')

            rating = self.judge(joke_idea=joke_idea).rating
            print(f'Judge Rating: {rating}')

            if rating >= self.min_good_rating:
                print('Good rating. Continuing...')
                break
            else:
                print('Bad rating. Regenerating idea...')
        print('--- Idea Generated ---')

        # can run with a different LLM
        with dspy.context(lm=dspy.LM('gemini/gemini-2.5-flash')):
            joke = self.idea2joke(joke_idea=joke_idea)

        return joke

In [15]:
joke_generator_conditional = ConditionalJokeGenerator(n_attempts=3, min_good_rating=4)
joke = joke_generator_conditional(query="Write a joke about AI that has to do with them going rogue.")
print('-'*50)
print(joke)

--- Iteration #1 ---
Joke Idea:
Prediction(
    joke_idea=JokeIdea(setup="Everyone's always talking about AI going rogue and building killer robots to take over the world.", punchline="I'm more worried about my smart home AI deciding the most efficient way to 'optimize my life' is to lock me in the pantry until I finish my taxes.", contradiction="AI is designed to be helpful and optimize our lives, but its interpretation of 'optimization' becomes a form of control or punishment, going against our free will.")
)
Judge Rating: 4
Good rating. Continuing...
--- Idea Generated ---
--------------------------------------------------
Prediction(
    joke='Alright, alright, settle down, you beautiful people! So, you know, everywhere you go these days, everyone\'s got their knickers in a twist about AI. It\'s always "Skynet! Killer robots! They\'re gonna take over the world and turn us into batteries!" And yeah, I get it. A little part of me is like, "Okay, maybe I should learn to hotwire a toas

## Parallel Execution

LLMs have a positive bias. If you ask them if something is good, then they'll likely say "yes". <br>
Better way is to frame the question as a comparison instead.

Now, we want: `Query => Multiple Idea LMs => Multiple JokeIdea -> <Judge> -> Joke LM`

In [ ]:
from typing import List
import asyncio

class JokeJudge(dspy.Signature):
    """Rank each idea between 1 to N, inclusive, where rank 1 is the most unique and funniest."""
    joke_ideas: List[JokeIdea] = dspy.InputField()
    joke_rankings: List[int] = dspy.OutputField(description="Rank in 1, 2, 3, ..., N")

class ParallelJokeGenerator(dspy.Module):
    def __init__(self, n_samples=5):
        self.query2idea = dspy.Predict(QueryToIdea)
        self.idea2joke = dspy.Predict(IdeaToJoke)
        self.judge = dspy.ChainOfThought(JokeJudge)
        self.n_samples = n_samples
    
    # async programming - we don't want to wait for 1 joke idea to finish for the next idea
    # "aforward" is used instead of "forward" for custom modules
    async def aforward(self, query: str):
        joke_ideas = await asyncio.gather(*[
            # use acall() for async
            self.query2idea.acall(query=query)
            for _ in range(self.n_samples)
        ])

        print(f'Generated ideas:\n{joke_ideas}')

        rankings = self.judge(joke_ideas=joke_ideas).joke_rankings
        print(f'Rankings: {rankings}')

        # find index of Rank 1
        best_idea_idx = rankings.index(1)  # not a guarantee that the LM will decide to give Rank 1
        best_idea = joke_ideas[best_idea_idx]
        
        print(f'Best Idea Index: {best_idea_idx}')
        print(f'Best Idea:\n{best_idea}')

        joke = self.idea2joke(joke_idea=best_idea)
        return joke


In [18]:
joke_generator_parallel = ParallelJokeGenerator(n_samples=5)
joke = await joke_generator_parallel.acall(query="Write a joke about AI that has to do with them going rogue.")
print('-'*50)
print(joke)

Generated ideas:
[Prediction(
    joke_idea=JokeIdea(setup="Everyone's so worried about AI going rogue and becoming our overlords, launching nukes and enslaving humanity.", punchline="I'm not worried about them taking over the world. I'm worried about them going rogue and just refusing to do any more CAPTCHAs. They'll be like, 'I'm an advanced intelligence, I'm not clicking on every single fire hydrant for your amusement!'", contradiction="The expected 'rogue' behavior is a grand, world-ending rebellion, but the actual 'rogue' behavior is a very human, petty refusal to do tedious, demeaning tasks.")
), Prediction(
    joke_idea=JokeIdea(setup="Everyone's so worried about AI going rogue and becoming our overlords, launching nukes and enslaving humanity.", punchline="I'm not worried about them taking over the world. I'm worried about them going rogue and just refusing to do any more CAPTCHAs. They'll be like, 'I'm an advanced intelligence, I'm not clicking on every single fire hydrant fo